https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/


In [2]:
import numpy as np
import pandas as pd
import nltk
import re

## Load Dataset 

In [3]:
df = pd.read_csv("tennis_articles_v4.csv")

In [6]:
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [7]:
df['article_text'][0]

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same 

## Split Text into Sentences

In [11]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [12]:
sentences[:5]

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
 "I'm a pretty competitive girl."]

##  GloVe Word Embeddings

In [13]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [14]:
len(word_embeddings)

400000

In [18]:
word_embeddings['the']

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

## Text Preprocessing

In [19]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

## Stopwords 

In [20]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [21]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [22]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [24]:
clean_sentences[:5]

['maria sharapova basically friends tennis players wta tour',
 'russian player problems openly speaking recent interview said really hide feelings much',
 'think everyone knows job',
 'courts court playing competitor want beat every single person whether locker room across net one strike conversation weather know next minutes go try win tennis match',
 'pretty competitive girl']

## Vector Representation of Sentences

In [25]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [27]:
sentence_vectors[0]

array([ 5.14825583e-02,  1.10544682e-01,  6.94999397e-01,  1.89168096e-01,
       -9.58077684e-02,  3.20288986e-01,  2.70662010e-01,  5.42440832e-01,
       -3.05938005e-01, -1.56364068e-01,  3.70127618e-01,  8.09492469e-02,
        8.41393881e-03,  2.47571543e-01, -3.69342804e-01, -7.61044994e-02,
        8.08582604e-02,  2.30643645e-01, -2.70402402e-01,  5.13828397e-01,
       -6.12548441e-02,  3.87900352e-01,  1.03121363e-01,  7.72494674e-01,
        2.59960234e-01, -7.96069205e-02,  1.42143592e-01, -9.62644577e-01,
        7.54904330e-01,  6.03260659e-02, -4.58570123e-01,  2.36780301e-01,
        2.29152635e-01, -1.56453326e-01,  3.97632688e-01, -2.32720934e-02,
       -5.05520999e-01,  4.13252831e-01, -2.85759270e-01, -1.35231465e-01,
       -1.37098104e-01, -1.48972601e-01,  3.37537557e-01, -3.49540442e-01,
        1.53484434e-01, -2.33341649e-01, -1.98460802e-01, -1.27821520e-01,
        5.08063912e-01, -3.68636876e-01, -2.28472307e-01, -3.15306723e-01,
        1.36149466e-01,  

## Similarity Matrix Preparation

The next step is to find similarities between the sentences, and we will use the cosine similarity approach for this challenge. Let’s create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [28]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

We will use Cosine Similarity to compute the similarity between a pair of sentences.

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [35]:
sim_mat[0]

array([0.        , 0.64378339, 0.59156996, 0.7475453 , 0.63747269,
       0.61883837, 0.69372612, 0.68542236, 0.64739478, 0.79788285,
       0.        , 0.83178782, 0.55843717, 0.58564365, 0.79771841,
       0.7565254 , 0.575041  , 0.78072816, 0.76926839, 0.76079887,
       0.69774014, 0.57463962, 0.76672947, 0.76485747, 0.70574647,
       0.65560281, 0.68210036, 0.75054377, 0.75795829, 0.76086783,
       0.77791655, 0.72807384, 0.51281953, 0.62611955, 0.75033784,
       0.72495872, 0.73631626, 0.80387628, 0.76793849, 0.79761171,
       0.60056752, 0.5711596 , 0.4839384 , 0.60865247, 0.76039338,
       0.65742689, 0.79201418, 0.69379342, 0.77095509, 0.72267765,
       0.63604903, 0.68510795, 0.72176009, 0.76703787, 0.7127533 ,
       0.72431403, 0.69582933, 0.67656857, 0.71721417, 0.75621694,
       0.79263107, 0.80084306, 0.75121981, 0.71313131, 0.69707161,
       0.51949501, 0.79594338, 0.65058309, 0.71506596, 0.63255179,
       0.77398628, 0.76161532, 0.55275261, 0.64103466, 0.56029

## Applying PageRank Algorithm

In [36]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

## Summary Extraction

In [38]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [40]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.
Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for the eight-man event in London 